# Constrained Graph Optimization (KOMO)

This is a dense optimization problem (in constrast to a path (k-order Markov) optimization). You can define arbitrary objectives across configurations.

In [1]:
import sys
sys.path.append('../ry')

from libry import *
from numpy import *

K = Config()
D = K.view()
K.addFile('../rai-robotModels/pr2/pr2.g')
K.addFile('../test/kitchen.g')

Let's add some objects

In [2]:
K.addFrame('item1', 'sink1', 'type:ssBox Q:<t(-.1 -.1 .52)> size:[.1 .1 .25 .02] color:[1. 0. 0.], contact' )
K.addFrame('item2', 'sink1', 'type:ssBox Q:<t(.1 .1 .52)> size:[.1 .1 .25 .02] color:[1. 1. 0.], contact' )
K.addFrame('tray', 'stove1', 'type:ssBox Q:<t(.0 .0 .42)> size:[.2 .2 .05 .02] color:[0. 1. 0.], contact' )

180

compute a collision free path to touch object item1 with pr2L:

In [3]:
obj1 = "item2";
obj2 = "item1";
tray = "tray";
arm = "pr2L";
table = "_12";

komo = K.komo_CGO(6)

komo.activateCollisionPairs([(obj1, obj2)]);
komo.addObjective([], OT.eq, FS.accumulatedCollisions);
komo.addObjective([], OT.ineq, FS.jointLimits);

komo.add_StableRelativePose([0, 1], arm, obj1);
komo.add_StableRelativePose([2, 3], arm, obj2);
komo.add_StableRelativePose([4, 5], arm, tray);

komo.add_StableRelativePose([1,2,3,4,5], tray, obj1);
komo.add_StableRelativePose([3,4,5], tray, obj2);

komo.add_StablePose([-1,0], obj1);
komo.add_StablePose([-1,0,1,2], obj2);
komo.add_StablePose([-1,0,1,2,3,4], tray);

komo.add_grasp(0, arm, obj1);
komo.add_place(1, obj1, tray);

komo.add_grasp(2, arm, obj2);
komo.add_place(3, obj2, tray);

komo.add_grasp(4, arm, tray);
komo.add_place(5, tray, table);

In [4]:
komo.optimize()

In [5]:
komo.displayTrajectory()

Best way to read out: grab a configuration into K and analyze it there:

In [12]:
K.setFrameState( komo.getConfiguration(4) )
K.getJointState()

array([  2.12658111e-01,   1.02870399e+00,   3.69820477e-01,
         1.48183819e-01,   1.02500341e-02,   9.37088085e-05,
        -1.01345867e+00,   1.47654561e+00,   4.07208297e-01,
         5.03687818e-01,   5.50273689e-01,  -1.00428534e+00,
         9.51999703e-01,  -1.98437937e+00,  -1.42265457e+00,
        -1.50297496e+00,   1.72005174e+00,  -4.82079837e-01,
        -5.34435335e-01,  -4.87929644e-01,   5.94821242e-01,
         8.24368284e-02,   8.88946887e-02,   1.27352254e-02,
         1.34174682e-02])